some part of this notebook are copied from Jose Marcial Portilla works.

# What is CUDA?

Most people confuse CUDA for a language or maybe an API. It is not.

It’s more than that. CUDA is a parallel computing platform and programming model that makes using a GPU for general purpose computing simple and elegant. The developer still programs in the familiar C, C++, Fortran, or an ever expanding list of supported languages, and incorporates extensions of these languages in the form of a few basic keywords.

These keywords let the developer express massive amounts of parallelism and direct the compiler to the portion of the application that maps to the GPU.

# How do I know if I have CUDA available?

In [ ]:
import torch
torch.cuda.is_available()
# True

True

In [ ]:
## Get Id of default device
torch.cuda.current_device()

0

In [ ]:
# 0
# torch.cuda.get_device_name(0) # Get name device with ID '0'
torch.cuda.get_device_name(torch.cuda.current_device())

'Tesla T4'

In [ ]:
# Returns the current GPU memory usage by 
# tensors in bytes for a given device
torch.cuda.memory_allocated()

0

In [ ]:
# Returns the current GPU memory managed by the
# caching allocator in bytes for a given device
torch.cuda.memory_cached()

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:393: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


0

# Using CUDA instead of CPU

In [ ]:
# CPU
a = torch.tensor([1.,2.])

In [ ]:
a

tensor([1., 2.])

In [ ]:
a.device

device(type='cpu')

In [ ]:
# GPU
a = torch.tensor([1., 2.]).cuda()

In [ ]:
a.device

device(type='cuda', index=0)

In [ ]:
torch.cuda.memory_allocated()

512

## Sending Models to GPU

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [ ]:
torch.manual_seed(32)
model = Model()

In [ ]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.3757, -0.2279, -0.0859,  0.2857],
        [-0.3870,  0.0793,  0.1481, -0.4771],
        [ 0.0874, -0.1746,  0.4485,  0.0219],
        [ 0.3782,  0.2254,  0.1929, -0.4741],
        [ 0.4319, -0.4087,  0.2177,  0.2271],
        [-0.0033,  0.4308, -0.1323, -0.2951],
        [ 0.1646, -0.1114, -0.4213, -0.3553],
        [-0.3191,  0.1440, -0.3496,  0.2280]], requires_grad=True)
Parameter containing:
tensor([ 0.3867, -0.2029,  0.2828,  0.0105,  0.3187, -0.0630, -0.3122,  0.3781],
       requires_grad=True)
Parameter containing:
tensor([[-0.2174,  0.0821,  0.2015, -0.2559, -0.3214,  0.1976, -0.3494, -0.2639],
        [ 0.1533, -0.1185,  0.2496, -0.0764,  0.0569, -0.1953,  0.0699, -0.3265],
        [-0.2689, -0.2716,  0.0416, -0.0628,  0.3519,  0.0132,  0.3048, -0.2101],
        [ 0.0877, -0.0607,  0.3204,  0.1180,  0.0254, -0.2260, -0.3002, -0.1128],
        [ 0.1822, -0.1912, -0.1485, -0.3094,  0.3335,  0.3195, -0.2034,  0.0114],
        [-0.0197,  0.0627, 

In [ ]:
# From the discussions here: discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda
next(model.parameters()).is_cuda

False

In [ ]:
gpumodel = model.cuda()

In [ ]:
next(gpumodel.parameters()).is_cuda

True

In [ ]:
# df = pd.read_csv('../content/Data/iris.csv')
# X = df.drop('target',axis=1).values
# y = df['target'].values
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

# df = pd.read_csv('../content/Data/iris.csv')
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)


## Convert Tensors to .cuda() tensors

In [ ]:
X_train = torch.tensor(X_train).cuda()
X_test = torch.tensor(X_test).cuda()
y_train = torch.tensor(y_train).cuda()
y_test = torch.tensor(y_test).cuda()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy 

In [ ]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.15071130
epoch: 11  loss: 0.93773133
epoch: 21  loss: 0.77982587
epoch: 31  loss: 0.60994011
epoch: 41  loss: 0.40079933
epoch: 51  loss: 0.25436324
epoch: 61  loss: 0.15053056
epoch: 71  loss: 0.10086948
epoch: 81  loss: 0.08128316
epoch: 91  loss: 0.07231428
TOTAL TRAINING TIME: 2.8728034496307373
